# Deep Learning - Exercise 1

## Covertype Data Set

Data set that was chosen for this exercise is the Covertype Data Set and the problem is a multiclass classification problem - predicting forest cover type from cartographic variables only (no remotely sensed data).

[Link to dataset in UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Covertype)

### Imports

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, MultiLabelBinarizer

from helper import clear_model, print_stats, get_batch

### Read the data

In [2]:
df = pd.read_csv('covtype.data', 
                 names = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 
                          'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 
                          'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 
                          'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1', 
                          'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4', 
                          'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 
                          'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 
                          'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 
                          'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 
                          'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 
                          'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 
                          'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 
                          'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 
                          'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 
                          'Soil_Type39', 'Soil_Type40', 'Cover_Type'])

X = df.loc[:, df.columns != 'Cover_Type'].astype(float)
y = df.loc[:, 'Cover_Type']

### Prepare the data as input for the network

In [3]:
# Normalize the data
X_values = X.iloc[:, :10].values.reshape(-1, 10)
X.iloc[:, :10] = MinMaxScaler().fit_transform(X_values)
# One hot encode labels
y = pd.DataFrame(MultiLabelBinarizer().fit_transform(y.values.reshape(-1, 1)).astype(float))

### Split the data into train, test and validation

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, 
                                                            test_size=0.1, random_state=42)

del X,y

### Hyperparameter tuning

In [5]:
epochs = 600
learning_rate = 0.1
keep_probability = 0.75
max_norm_constraint = 3
regularizer_type = 'L2'
regularization = 0.01
hidden_layer_size = 512
batch_size = 32

### Build the network

In [6]:
# Reset
tf.reset_default_graph()

# Placeholders
x = tf.placeholder(tf.float32, [None, len(X_train.columns)], 'x')
y = tf.placeholder(tf.float32, [None, len(y_train.columns)], 'y')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

# Regularization
regularizer = None
if regularizer_type == 'L1':
    regularizer = tf.contrib.layers.l1_regularizer(regularization)
elif regularizer_type == 'L2':
    regularizer = tf.contrib.layers.l2_regularizer(regularization)
#TODO - max norm constraint?

# Model
model = tf.contrib.layers.fully_connected(x, hidden_layer_size, tf.nn.relu, 
                                          weights_regularizer=regularizer)
model = tf.nn.dropout(model, keep_prob)
model = tf.contrib.layers.fully_connected(model, hidden_layer_size, tf.nn.relu, 
                                          weights_regularizer=regularizer)
model = tf.nn.dropout(model, keep_prob)
model = tf.contrib.layers.fully_connected(model, len(y_train.columns), 
                                          weights_regularizer=regularizer)
logits = tf.nn.softmax(model)

# Loss and Optimizer
loss = tf.reduce_mean(tf.squared_difference(logits, y))
tf.summary.scalar('loss', loss)
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')
tf.summary.scalar('acc', accuracy)

summary_op = tf.summary.merge_all()

### For saving the model and writing to Tensorboard

In [7]:
save_model_path = 'model/'
model_name = 'model'
tensorboard_path = 'tensorboard/'

clear_model(save_model_path, tensorboard_path)

train_summary_writer = tf.summary.FileWriter(tensorboard_path + 'train')
test_summary_writer = tf.summary.FileWriter(tensorboard_path + 'test')

saver = tf.train.Saver()

### Training the model

In [8]:
print 'Training...'

start = datetime.datetime.now()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        n_batches = len(X_train.index) / batch_size
        for batch_i in range(n_batches):
            batch_features, batch_labels = get_batch(X_train, y_train, batch_i, batch_size)
            sess.run(optimizer, feed_dict={ x: batch_features, y: batch_labels, 
                                            keep_prob: keep_probability })
        print_stats(sess, epoch, X_train, y_train, X_validate, y_validate, loss, accuracy, 
                    x, y, keep_prob, summary_op, train_summary_writer, test_summary_writer)
        saver.save(sess, save_model_path + model_name, global_step=epoch)
    saver.save(sess, save_model_path + model_name)  
total = datetime.datetime.now() - start

print 'Training complete\nTotal time: %.2f' % total.total_seconds() + ' seconds'

Training...
Epoch  1: loss: 0.0596 accuracy: 0.707
Epoch  2: loss: 0.0570 accuracy: 0.719
Epoch  3: loss: 0.0557 accuracy: 0.724
Epoch  4: loss: 0.0532 accuracy: 0.741
Epoch  5: loss: 0.0518 accuracy: 0.746
Epoch  6: loss: 0.0510 accuracy: 0.751
Epoch  7: loss: 0.0496 accuracy: 0.755
Epoch  8: loss: 0.0485 accuracy: 0.761
Epoch  9: loss: 0.0476 accuracy: 0.765
Epoch 10: loss: 0.0466 accuracy: 0.771
Epoch 11: loss: 0.0459 accuracy: 0.775
Epoch 12: loss: 0.0451 accuracy: 0.780
Epoch 13: loss: 0.0444 accuracy: 0.783
Epoch 14: loss: 0.0439 accuracy: 0.786
Epoch 15: loss: 0.0433 accuracy: 0.790
Epoch 16: loss: 0.0426 accuracy: 0.795
Epoch 17: loss: 0.0421 accuracy: 0.796
Epoch 18: loss: 0.0415 accuracy: 0.800
Epoch 19: loss: 0.0409 accuracy: 0.804
Epoch 20: loss: 0.0405 accuracy: 0.804
Epoch 21: loss: 0.0399 accuracy: 0.809
Epoch 22: loss: 0.0395 accuracy: 0.810
Epoch 23: loss: 0.0390 accuracy: 0.812
Epoch 24: loss: 0.0386 accuracy: 0.815
Epoch 25: loss: 0.0381 accuracy: 0.817
Epoch 26: los

Epoch 208: loss: 0.0184 accuracy: 0.912
Epoch 209: loss: 0.0185 accuracy: 0.911
Epoch 210: loss: 0.0185 accuracy: 0.912
Epoch 211: loss: 0.0184 accuracy: 0.912
Epoch 212: loss: 0.0183 accuracy: 0.913
Epoch 213: loss: 0.0183 accuracy: 0.912
Epoch 214: loss: 0.0183 accuracy: 0.913
Epoch 215: loss: 0.0183 accuracy: 0.912
Epoch 216: loss: 0.0182 accuracy: 0.913
Epoch 217: loss: 0.0182 accuracy: 0.912
Epoch 218: loss: 0.0181 accuracy: 0.913
Epoch 219: loss: 0.0182 accuracy: 0.912
Epoch 220: loss: 0.0181 accuracy: 0.913
Epoch 221: loss: 0.0180 accuracy: 0.914
Epoch 222: loss: 0.0181 accuracy: 0.912
Epoch 223: loss: 0.0178 accuracy: 0.915
Epoch 224: loss: 0.0180 accuracy: 0.913
Epoch 225: loss: 0.0181 accuracy: 0.914
Epoch 226: loss: 0.0178 accuracy: 0.915
Epoch 227: loss: 0.0178 accuracy: 0.914
Epoch 228: loss: 0.0177 accuracy: 0.914
Epoch 229: loss: 0.0177 accuracy: 0.914
Epoch 230: loss: 0.0178 accuracy: 0.914
Epoch 231: loss: 0.0176 accuracy: 0.914
Epoch 232: loss: 0.0177 accuracy: 0.915


Epoch 413: loss: 0.0140 accuracy: 0.930
Epoch 414: loss: 0.0140 accuracy: 0.929
Epoch 415: loss: 0.0140 accuracy: 0.929
Epoch 416: loss: 0.0139 accuracy: 0.931
Epoch 417: loss: 0.0140 accuracy: 0.929
Epoch 418: loss: 0.0139 accuracy: 0.930
Epoch 419: loss: 0.0139 accuracy: 0.930
Epoch 420: loss: 0.0139 accuracy: 0.930
Epoch 421: loss: 0.0139 accuracy: 0.929
Epoch 422: loss: 0.0139 accuracy: 0.929
Epoch 423: loss: 0.0139 accuracy: 0.930
Epoch 424: loss: 0.0140 accuracy: 0.930
Epoch 425: loss: 0.0139 accuracy: 0.930
Epoch 426: loss: 0.0139 accuracy: 0.929
Epoch 427: loss: 0.0136 accuracy: 0.930
Epoch 428: loss: 0.0137 accuracy: 0.930
Epoch 429: loss: 0.0137 accuracy: 0.931
Epoch 430: loss: 0.0138 accuracy: 0.931
Epoch 431: loss: 0.0138 accuracy: 0.930
Epoch 432: loss: 0.0138 accuracy: 0.930
Epoch 433: loss: 0.0137 accuracy: 0.930
Epoch 434: loss: 0.0137 accuracy: 0.931
Epoch 435: loss: 0.0137 accuracy: 0.931
Epoch 436: loss: 0.0136 accuracy: 0.929
Epoch 437: loss: 0.0136 accuracy: 0.931


### Test the model

In [9]:
print 'Testing...'

loaded_graph = tf.Graph()

with tf.Session(graph=loaded_graph) as sess:
    loader = tf.train.import_meta_graph(save_model_path + model_name + '.meta')
    loader.restore(sess, save_model_path + model_name)
    loaded_x = loaded_graph.get_tensor_by_name('x:0')
    loaded_y = loaded_graph.get_tensor_by_name('y:0')
    loaded_keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    loaded_acc = loaded_graph.get_tensor_by_name('accuracy:0')
    test_batch_acc_total = 0
    test_batch_count = 0
    n_batches = len(X_test.index) / batch_size
    for batch_i in range(n_batches):
        batch_features, batch_labels = get_batch(X_test, y_test, batch_i, batch_size)
        test_batch_acc_total += sess.run(loaded_acc, feed_dict={ loaded_x: batch_features, 
                                                     loaded_y: batch_labels, 
                                                     loaded_keep_prob: 1.0 })
        test_batch_count += 1
    print 'Testing Accuracy: %.2f' % (test_batch_acc_total / test_batch_count)

Testing...
INFO:tensorflow:Restoring parameters from model/model
Testing Accuracy: 0.94
